# Sensorless Drive Diagnosis

> In this example, the main focus is the classification of individual states of a motor.

In [ ]:
# |hide
from nbdev.showdoc import *

In [ ]:
# | hide
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd
import copy

from sklearn.metrics import classification_report
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from matplotlib import pyplot as plt
import seaborn as sns

from mlmvn.layers import FirstLayer, HiddenLayer, OutputLayer, cmplx_phase_activation
from mlmvn.loss import ComplexMSELoss, ComplexMSE_adjusted_error
from mlmvn.optim import MySGD, ECL
from pathlib import Path
from clearml import Task, Logger

In [ ]:
# | hide
# --- helper functions ---
def reverse_one_hot(x, neuronCats):
    a = np.zeros(len(x))
    x = torch.detach(x)
    for i in range(len(x)):
        a[i] = torch.max(x[i]) - 1 + np.argmax(x[i]) * neuronCats
    return a


def accuracy(out, yb):
    out = out.type(torch.double)
    yb = yb.type(torch.double)
    x = 0
    for i in range(len(out)):
        x += torch.equal(out[i], yb[i])
    return x / len(out)


def prepare_data(x_train, x_valid, y_train, y_valid, neuronCats):
    # one-hot encoding
    numSamples, numFeatures = x_valid.shape
    y_valid_int = y_valid
    y2 = y_valid + 1  # auxiliary variable so that classes start at 1 and not 0
    numClasses = max(y2)
    target_ids = range(numClasses)
    no = int(np.ceil(numClasses / neuronCats))  # number of output neurons
    if no != 1:
        y_valid = torch.zeros(numSamples, no)
        for i in range(numSamples):
            k = int(np.ceil(y2[i] / neuronCats)) - 1
            c = np.mod((y2[i] - 1), neuronCats) + 1
            y_valid[i, k] = c
    numSamples, numFeatures = x_train.shape
    y_train_int = y_train
    y2 = y_train + 1  # auxiliary variable so that classes start at 1 and not 0
    if no != 1:
        y_train = torch.zeros(numSamples, no)
        for i in range(numSamples):
            k = int(np.ceil(y2[i] / neuronCats)) - 1
            c = np.mod((y2[i] - 1), neuronCats) + 1
            y_train[i, k] = c
    del y2

    # Convert numpy arrays into torch tensors
    x_train, y_train, x_valid, y_valid = map(
        torch.tensor, (x_train, y_train, x_valid, y_valid)
    )
    if y_train.size().__len__() == 1:
        y_train = torch.unsqueeze(y_train, 1)
        y_valid = torch.unsqueeze(y_valid, 1)

    # convert angles to complex numbers on unit-circle
    x_train = torch.exp(1.0j * x_train)
    x_valid = torch.exp(1.0j * x_valid)

    return x_train, x_valid, y_train, y_valid


def get_splitted_data(X, y, neuronCats):
    x_train, x_valid, y_train, y_valid = train_test_split(
        X, y, train_size=46806, random_state=42
    )
    x_train, x_valid, y_train, y_valid = prepare_data(
        x_train, x_valid, y_train, y_valid, neuronCats
    )

    return x_train, x_valid, y_train, y_valid


def get_splitted_data_by_index(X, y, neuronCats, train_index, test_index):
    x_train, x_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    x_train, x_valid, y_train, y_valid = prepare_data(
        x_train, x_valid, y_train, y_valid, neuronCats
    )
    return x_train, x_valid, y_train, y_valid


# --- Plots ---
def plot_loss(title, losses, scores):
    plt.rcParams["axes.grid"] = True
    fig, (ax1) = plt.subplots(1, 1, figsize=(8, 4))
    fig.suptitle("CVNN - Moons")
    ax1.plot(np.linspace(1, len(losses), len(losses)), losses)
    ax1.set_xlabel("Epoch")
    ax1.set_xlim(0, len(losses))

    ax1.plot(np.linspace(1, len(scores), len(scores)), scores)
    ax1.set_xlabel("Epoch")
    ax1.set_xlim(0, len(losses))

    ax1.legend(["Acc", "Loss"])

    plt.show()


def plot_weights(title, ylabel_1, ylabel_2, weights_real, weights_imag):
    # y_min = np.min([np.min(weights_real), np.min(weights_imag)])
    # y_max = np.max([np.max(weights_real), np.max(weights_imag)])

    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(14, 3))
    fig.suptitle(title)
    ax[0].plot(np.linspace(1, len(weights_real), len(weights_real)), weights_real)
    ax[0].set_xlabel("Step")
    ax[0].set_ylabel(ylabel_1)
    # ax[0].set_title("Real Valued Weigts")
    ax[0].set_xlim(0, len(weights_real))
    # ax[0].set_ylim(y_min, y_max)

    ax[1].plot(np.linspace(1, len(weights_imag), len(weights_imag)), weights_imag)
    ax[1].set_xlabel("Step")
    ax[1].set_ylabel(ylabel_2)
    # ax[1].set_title("Imaginary Valued Weights")
    ax[1].set_xlim(0, len(weights_imag))
    # ax[1].set_ylim(y_min, y_max)

    plt.show()


def plot_loss_acc_list(title, list_losses, list_scores, image_name):
    losses = np.mean(list_losses, axis=0)
    scores = np.mean(list_scores, axis=0)

    losses_std = np.std(list_losses, axis=0)
    scores_std = np.std(list_scores, axis=0)

    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 3))
    fig.suptitle(title)
    ax1.plot(np.linspace(1, len(losses), len(losses)), losses)
    ax1.fill_between(
        np.linspace(1, len(losses), len(losses)),
        losses + losses_std,
        losses - losses_std,
        alpha=0.5,
        linewidth=0,
    )

    ax1.plot(np.linspace(1, len(scores), len(scores)), scores)
    ax1.fill_between(
        np.linspace(1, len(scores), len(scores)),
        scores + scores_std,
        scores - scores_std,
        alpha=0.5,
        linewidth=0,
    )
    ax1.set_xlabel("Epoch")

    plt.legend(["Loss Mean", "Loss Std", "Acc. Mean", "Acc. Std"])
    fig.savefig(image_name, format="png", dpi=600)

    plt.show()
    # save
    # fig.savefig(image_name + ".svg", format="svg", dpi=600)


# --- Logging ---
model_dict: dict = {}


def fc_hook(layer_name, module, grad_input, grad_output):
    if layer_name in model_dict:
        model_dict[layer_name]["weights"] = module.weights.detach().clone()
        model_dict[layer_name]["bias"] = module.bias.detach().clone()
        model_dict[layer_name]["grad_input"] = grad_input
        model_dict[layer_name]["grad_output"] = grad_output
    else:
        model_dict[layer_name] = {}
        model_dict[layer_name]["weights"] = module.weights.detach().clone()
        model_dict[layer_name]["bias"] = module.bias.detach().clone()
        model_dict[layer_name]["grad_input"] = grad_input
        model_dict[layer_name]["grad_output"] = grad_output

In [ ]:
# | hide
# control variables
# number of categories a neuron can distinguish / parameter that determines the number of output neurons
neuronCats = 1
# number of categories per neuron, i.e. neuronCats (+ 1 for others in case of multiple Outputs)
categories = 2
# how often a classification sector occurs (1 means no periodicity)
periodicity = 1
# path to store best model parameters

## Load Data

In [ ]:
train_csv = pd.read_csv(
    "data/autass_data2.csv",
    header=None,
    dtype=np.double,
)
data = np.array(train_csv.values[:, 1:50])
del train_csv

In [ ]:
X = data[:, 0:48]
y = data[:, 48].astype(int) - 1

yt = copy.copy(y)
yt[yt == 0] = 20
yt[yt == 1] = 21
yt[yt == 2] = 22
yt[yt == 3] = 23
yt[yt == 4] = 26
yt[yt == 5] = 24
yt[yt == 6] = 27
yt[yt == 7] = 29
yt[yt == 8] = 30
yt[yt == 9] = 25
yt[yt == 10] = 28
yt -= 20
y = yt
del yt

## Config

In [ ]:
epochs = 200
batch_size = 538
lr = 1
clip_angle_value = 1000000

## Single Layer

### MLMVN [48-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=854e9bc7dc7e496ea841a13f617da653
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/854e9bc7dc7e496ea841a13f617da653/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 14:25:57,872 - clearml.frameworks - INFO - Found existing registered model id=caa96da5a415490ca1ea0f95b383f403 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-11.pt] reusing it.
Epoch 9 loss is 0.312786317016275
Epoch 19 loss is 0.25568547822538934
Epoch 29 loss is 0.3389669520974066
Epoch 39 loss is 0.28586026849296975
Epoch 49 loss is 0.2642590965695329
Epoch 59 loss is 0.2561917205608428
Epoch 69 loss is 0.2538907377290161
Epoch 79 loss is 0.25494624541893784
Epoch 89 loss is 0.2586601196421547
Epoch 99 loss is 0.26136219948247824
Epoch 109 loss is 0.25920406138033625
Epoch 119 loss is 0.257771638513778
Epoch 129 loss is 0.256717965344817
Epoch 139 loss is 0.2562337465502512
Epoch 149 loss is 0.25599395140123643
Epoch 159 loss is 0.2555283782128066
Epoch 169 loss is 0.25500644305516845
Epoch 179 loss is 0.2546311936985765
Epoch 189 loss is 0.2545335270219525
Epoch 199 loss is 0.2544423936517196
Train Acc.:  0.7605486358877945
            

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.23875833944773336
Epoch 19 loss is 0.35046091409537855
Epoch 29 loss is 0.3474688264639837
Epoch 39 loss is 0.3460845462685393
Epoch 49 loss is 0.34521615010787904
Epoch 59 loss is 0.34480239729514767
Epoch 69 loss is 0.3445531601110381
Epoch 79 loss is 0.344414452281505
Epoch 89 loss is 0.3443463357528307
Epoch 99 loss is 0.34433482782616937
Epoch 109 loss is 0.3443202594365852
Epoch 119 loss is 0.3443419606632198
Epoch 129 loss is 0.34439574951313695
Epoch 139 loss is 0.34447653188306476
Epoch 149 loss is 0.34455745559313744
Epoch 159 loss is 0.3446490512911633
Epoch 169 loss is 0.34472025490019265
Epoch 179 loss is 0.34483666034343385
Epoch 189 loss is 0.3449189385351294
Epoch 199 loss is 0.34502282438946064
Train Acc.:  0.7681329715640823
              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1063
           1       0.88      0.68      0.77      1064
           2       0.90      0.73      0.81      1064
           3

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.2971094108173737
Epoch 19 loss is 0.3869108816673947
Epoch 29 loss is 0.4365067504356039
Epoch 39 loss is 0.4177013765514301
Epoch 49 loss is 0.4124413908694452
Epoch 59 loss is 0.40500117037391986
Epoch 69 loss is 0.41131196047303786
Epoch 79 loss is 0.4181249931351551
Epoch 89 loss is 0.41661642708626356
Epoch 99 loss is 0.4181854146942287
Epoch 109 loss is 0.4133348665799685
Epoch 119 loss is 0.4099116159030149
Epoch 129 loss is 0.4199283736599332
Epoch 139 loss is 0.417771504949945
Epoch 149 loss is 0.4188665644250296
Epoch 159 loss is 0.42150588067988737
Epoch 169 loss is 0.4230460352811647
Epoch 179 loss is 0.4250528931966397
Epoch 189 loss is 0.4252225452870114
Epoch 199 loss is 0.42203162628683427
Train Acc.:  0.7921464738180187
              precision    recall  f1-score   support

           0       0.95      0.88      0.92      1064
           1       0.85      0.66      0.75      1064
           2       0.92      0.80      0.86      1064
           3      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.2795195632281697
Epoch 19 loss is 0.3375428780831629
Epoch 29 loss is 0.33256474318305307
Epoch 39 loss is 0.33185831863511417
Epoch 49 loss is 0.3295307025439074
Epoch 59 loss is 0.3210891922650079
Epoch 69 loss is 0.32034934104938245
Epoch 79 loss is 0.3163362890807787
Epoch 89 loss is 0.315736489078309
Epoch 99 loss is 0.3155383143134653
Epoch 109 loss is 0.31553327463043673
Epoch 119 loss is 0.31600845927522986
Epoch 129 loss is 0.3150536779122853
Epoch 139 loss is 0.3150594287707738
Epoch 149 loss is 0.31524582921067934
Epoch 159 loss is 0.3153440366348357
Epoch 169 loss is 0.31513214139197515
Epoch 179 loss is 0.3152703075228755
Epoch 189 loss is 0.3215461540110862
Epoch 199 loss is 0.31620214863132784
Train Acc.:  0.7109833999188155
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      1063
           1       0.84      0.76      0.80      1064
           2       0.82      0.71      0.76      1064
           3   

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.39677247203032356
Epoch 19 loss is 0.35580593764225016
Epoch 29 loss is 0.42506711288150867
Epoch 39 loss is 0.4258715088317278
Epoch 49 loss is 0.42243423247066575
Epoch 59 loss is 0.4237868170495762
Epoch 69 loss is 0.41925568150605863
Epoch 79 loss is 0.439859722214063
Epoch 89 loss is 0.4537169174860011
Epoch 99 loss is 0.4516644332667944
Epoch 109 loss is 0.4505254539904648
Epoch 119 loss is 0.44988270158956056
Epoch 129 loss is 0.4494370519612547
Epoch 139 loss is 0.449528904207753
Epoch 149 loss is 0.44943834853116904
Epoch 159 loss is 0.449371091649806
Epoch 169 loss is 0.4491654165153552
Epoch 179 loss is 0.4491931206111527
Epoch 189 loss is 0.44876226191301677
Epoch 199 loss is 0.44890430281994864
Train Acc.:  0.6543465720939176
              precision    recall  f1-score   support

           0       0.70      0.86      0.77      1064
           1       0.00      0.00      0.00      1064
           2       0.95      0.68      0.79      1064
           3    

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=47852b7bd30c475a9fbb69e5802aea36
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/47852b7bd30c475a9fbb69e5802aea36/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 14:42:43,693 - clearml.frameworks - INFO - Found existing registered model id=c337b94a22444d809d449783726d8ee2 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-11.pt] reusing it.
Epoch 9 loss is 0.15395791001477882
Epoch 19 loss is 0.15815942352420895
Epoch 29 loss is 0.1521475958209485
Epoch 39 loss is 0.22264992876002446
Epoch 49 loss is 0.2434950175800985
Epoch 59 loss is 0.3000683165649394
Epoch 69 loss is 0.2986102485389353
Epoch 79 loss is 0.2978965348099431
Epoch 89 loss is 0.2976461313968712
Epoch 99 loss is 0.297615187920047
Epoch 109 loss is 0.2975814279553468
Epoch 119 loss is 0.2973692060082367
Epoch 129 loss is 0.2971646093759705
Epoch 139 loss is 0.2969823604569539
Epoch 149 loss is 0.2968226945940804
Epoch 159 loss is 0.29669339181528953
Epoch 169 loss is 0.2965860987914452
Epoch 179 loss is 0.2964960589856526
Epoch 189 loss is 0.296397997745145
Epoch 199 loss is 0.2963285508675669
Train Acc.:  0.8742068494028671
              

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.23371587049756218
Epoch 19 loss is 0.2171053793431578
Epoch 29 loss is 0.2117481726413607
Epoch 39 loss is 0.20548662762905423
Epoch 49 loss is 0.20055165801015304
Epoch 59 loss is 0.19926006465921117
Epoch 69 loss is 0.20121967169245297
Epoch 79 loss is 0.2012524994483403
Epoch 89 loss is 0.2089158789663104
Epoch 99 loss is 0.2069847307533938
Epoch 109 loss is 0.20158804575222558
Epoch 119 loss is 0.19908199121184547
Epoch 129 loss is 0.1983263363973267
Epoch 139 loss is 0.1981246632369139
Epoch 149 loss is 0.19843244794655257
Epoch 159 loss is 0.19990613128620335
Epoch 169 loss is 0.20346991661240654
Epoch 179 loss is 0.20220646457032931
Epoch 189 loss is 0.20162040661533578
Epoch 199 loss is 0.20261586962160896
Train Acc.:  0.7864421988164163
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1063
           1       0.90      0.78      0.84      1064
           2       0.98      0.66      0.79      1064
         

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.17359070471532762
Epoch 19 loss is 0.19923888591987557
Epoch 29 loss is 0.22116873452047683
Epoch 39 loss is 0.2029670658449249
Epoch 49 loss is 0.20419508592690006
Epoch 59 loss is 0.19794629333818395
Epoch 69 loss is 0.1952743740865991
Epoch 79 loss is 0.19510413114878267
Epoch 89 loss is 0.19646997157487037
Epoch 99 loss is 0.19606822933983284
Epoch 109 loss is 0.19653955121411834
Epoch 119 loss is 0.19678779104367372
Epoch 129 loss is 0.19675274303142729
Epoch 139 loss is 0.19700162446357028
Epoch 149 loss is 0.1971595217735474
Epoch 159 loss is 0.19742537431793827
Epoch 169 loss is 0.19757406298027266
Epoch 179 loss is 0.1976568692427292
Epoch 189 loss is 0.19808705126678153
Epoch 199 loss is 0.19829968689615954
Train Acc.:  0.8086610976990621
              precision    recall  f1-score   support

           0       0.93      0.90      0.91      1064
           1       0.87      0.82      0.85      1064
           2       0.97      0.91      0.94      1064
      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.16241861081194287
Epoch 19 loss is 0.142375697260239
Epoch 29 loss is 0.15385840834181844
Epoch 39 loss is 0.2138264016307598
Epoch 49 loss is 0.2365240586587247
Epoch 59 loss is 0.2929398953187324
Epoch 69 loss is 0.2761381170166951
Epoch 79 loss is 0.3456885180336249
Epoch 89 loss is 0.34346247018042997
Epoch 99 loss is 0.3458485532239313
Epoch 109 loss is 0.3444044925757802
Epoch 119 loss is 0.3440097919960899
Epoch 129 loss is 0.34428306482555965
Epoch 139 loss is 0.3449518206799149
Epoch 149 loss is 0.3461375016053481
Epoch 159 loss is 0.3476872522585145
Epoch 169 loss is 0.34796445004321847
Epoch 179 loss is 0.3482418109346652
Epoch 189 loss is 0.34871055688690134
Epoch 199 loss is 0.3490575960919955
Train Acc.:  0.8840558036191168
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1063
           1       0.92      0.90      0.91      1064
           2       0.97      0.93      0.95      1064
           3     

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.15193523247470855
Epoch 19 loss is 0.1408373010379282
Epoch 29 loss is 0.18472082419796915
Epoch 39 loss is 0.1959035029241411
Epoch 49 loss is 0.21619177741693082
Epoch 59 loss is 0.21511564006011144
Epoch 69 loss is 0.20399205995454375
Epoch 79 loss is 0.20022756204760384
Epoch 89 loss is 0.20033638712455112
Epoch 99 loss is 0.19893451606025606
Epoch 109 loss is 0.19786037468191467
Epoch 119 loss is 0.19890569036312059
Epoch 129 loss is 0.1990134917790664
Epoch 139 loss is 0.19872859569206974
Epoch 149 loss is 0.19751614804725498
Epoch 159 loss is 0.19739665962479105
Epoch 169 loss is 0.19784372444886006
Epoch 179 loss is 0.19111926939075988
Epoch 189 loss is 0.19073564886846614
Epoch 199 loss is 0.18976225840289762
Train Acc.:  0.8771764906958361
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1064
           1       0.89      0.79      0.84      1064
           2       0.97      0.93      0.95      1064
     

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=0e2885b9c6934f79849153fe8c7149d5
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/0e2885b9c6934f79849153fe8c7149d5/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 14:56:58,474 - clearml.frameworks - INFO - Found existing registered model id=bb96e63090904339bf87c4852d30bdb6 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-11.pt] reusing it.
Epoch 9 loss is 0.11677723785531834
Epoch 19 loss is 0.08727784719556735
Epoch 29 loss is 0.09157292581128637
Epoch 39 loss is 0.07421681373324716
Epoch 49 loss is 0.07697537707769428
Epoch 59 loss is 0.06723158685548768
Epoch 69 loss is 0.06324904770714807
Epoch 79 loss is 0.0635766536591303
Epoch 89 loss is 0.0689652033514836
Epoch 99 loss is 0.09650615274908997
Epoch 109 loss is 0.07453832805905608
Epoch 119 loss is 0.07929171747710538
Epoch 129 loss is 0.07981405980555767
Epoch 139 loss is 0.09314429308579714
Epoch 149 loss is 0.1007599468837455
Epoch 159 loss is 0.1594258365172202
Epoch 169 loss is 0.16438126926324473
Epoch 179 loss is 0.1535026421169871
Epoch 189 loss is 0.15247239398711698
Epoch 199 loss is 0.1501105166766776
Train Acc.:  0.9464609994231632
  

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.10269231647718571
Epoch 19 loss is 0.0828424365293163
Epoch 29 loss is 0.0755315495091804
Epoch 39 loss is 0.07214960396708685
Epoch 49 loss is 0.06101909084585404
Epoch 59 loss is 0.06839280821097835
Epoch 69 loss is 0.0599125453507835
Epoch 79 loss is 0.07335060013207284
Epoch 89 loss is 0.07783843407293703
Epoch 99 loss is 0.07579400460243982
Epoch 109 loss is 0.08774453313883374


In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=7b3abe9fb989495e892c444570be6067
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/7b3abe9fb989495e892c444570be6067/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_2584/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 19:35:24,268 - clearml.frameworks - INFO - Found existing registered model id=0f73e6db01fc42988672e4f44c0add5f [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-11.pt] reusing it.
Epoch 9 loss is 0.09364351451345408
Epoch 19 loss is 0.06047805760727481
Epoch 29 loss is 0.0578744128569904
Epoch 39 loss is 0.04534195824524318
Epoch 49 loss is 0.04307949224899972
Epoch 59 loss is 0.041017087963280054
Epoch 69 loss is 0.03752497958328672
Epoch 79 loss is 0.03929154410570669
Epoch 89 loss is 0.03530748449555099
Epoch 99 loss is 0.03361906166904868
Epoch 109 loss is 0.03500346362768258
Epoch 119 loss is 0.03421256406705508
Epoch 129 loss is 0.03967580897383263
Epoch 139 loss is 0.03983386888202658
Epoch 149 loss is 0.035099183909890475
Epoch 159 loss is 0.039366477472969064
Epoch 169 loss is 0.039393422961427775
Epoch 179 loss is 0.03634864197734159
Epoch 189 loss is 0.03297036735253193
Epoch 199 loss is 0.03543338693745545
Train Acc.:  0.974447720

In [ ]:
task.mark_completed()
task.close()

## Multi Layer

### MLMVN [48-10-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(10, 10)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-10-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-10-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=9f8e1bb1a8ce4055b17fd15ff8f5f96e
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/9f8e1bb1a8ce4055b17fd15ff8f5f96e/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-10-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_2584/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 19:47:37,678 - clearml.frameworks - INFO - Found existing registered model id=410edb2915b24269b7d34f2e38593dff [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-10-11.pt] reusing it.
Epoch 9 loss is 0.3190616793468566
Epoch 19 loss is 0.3316516310882961
Epoch 29 loss is 0.3159845589401083
Epoch 39 loss is 0.2897936458677712
Epoch 49 loss is 0.29438441483606387
Epoch 59 loss is 0.29755967444288156
Epoch 69 loss is 0.2976103369922987
Epoch 79 loss is 0.3521991805852908
Epoch 89 loss is 0.3175364303332033
Epoch 99 loss is 0.3012195794930889
Epoch 109 loss is 0.3188232513425489
Epoch 119 loss is 0.32362149784936967
Epoch 129 loss is 0.3005814172908205
Epoch 139 loss is 0.2813095426634295
Epoch 149 loss is 0.26490572151648945
Epoch 159 loss is 0.311720634966309
Epoch 169 loss is 0.2963948818829981
Epoch 179 loss is 0.2921389037327001
Epoch 189 loss is 0.3533133125418626
Epoch 199 loss is 0.3518870098072943
Train Acc.:  0.7671666025723198
Val Acc.: 

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(20, 20)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-20-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-20-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=1b1a786365114896aa361f7567b2a590
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/1b1a786365114896aa361f7567b2a590/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-20-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_2584/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 19:56:38,160 - clearml.frameworks - INFO - Found existing registered model id=22ba5a4169ed406a9e74f40200bd29a1 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-20-11.pt] reusing it.
Epoch 9 loss is 0.22423453263455123
Epoch 19 loss is 0.2417288273187978
Epoch 29 loss is 0.24098897281491427
Epoch 39 loss is 0.25785384004717277
Epoch 49 loss is 0.20411375730867218
Epoch 59 loss is 0.19907421010282828
Epoch 69 loss is 0.21536965079441342
Epoch 79 loss is 0.21283156041274492
Epoch 89 loss is 0.20860426362519324
Epoch 99 loss is 0.1986070523113828
Epoch 109 loss is 0.21692347488143257
Epoch 119 loss is 0.23349276288092297
Epoch 129 loss is 0.21625116723365828
Epoch 139 loss is 0.23410189520133975
Epoch 149 loss is 0.23888556577670042
Epoch 159 loss is 0.23277480097426984
Epoch 169 loss is 0.20190034595150705
Epoch 179 loss is 0.20968810270215624
Epoch 189 loss is 0.2009444427805874
Epoch 199 loss is 0.2018091584845151
Train Acc.:  0.84085373670042

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(50, 50)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-50-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-50-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=88a9a42920894ee480fda889218b6295
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/88a9a42920894ee480fda889218b6295/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-50-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_2584/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 20:06:38,774 - clearml.frameworks - INFO - Found existing registered model id=f13061c5d03a4e96b788becd5e54443a [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-50-11.pt] reusing it.
Epoch 9 loss is 0.12497814221423068
Epoch 19 loss is 0.11329098739263005
Epoch 29 loss is 0.1134584198279062
Epoch 39 loss is 0.10747780190630066
Epoch 49 loss is 0.10495547213762008
Epoch 59 loss is 0.10659716730495539
Epoch 69 loss is 0.09905256375984144
Epoch 79 loss is 0.09258128419659628
Epoch 89 loss is 0.08945538147457059
Epoch 99 loss is 0.10534673199439117
Epoch 109 loss is 0.09873397783198988
Epoch 119 loss is 0.0957623519177352
Epoch 129 loss is 0.1089078274844085
Epoch 139 loss is 0.10761467200030443
Epoch 149 loss is 0.0974923953101792
Epoch 159 loss is 0.10399220587872246
Epoch 169 loss is 0.11563031956286923
Epoch 179 loss is 0.1356005221774724
Epoch 189 loss is 0.11880395204472788
Epoch 199 loss is 0.11979121066603561
Train Acc.:  0.934709225312994

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(100, 100)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-100-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": lr,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-100-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=d9b3caa55d294da8b5913a5161413f86
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/d9b3caa55d294da8b5913a5161413f86/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-100-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSE_adjusted_error.apply
    optimizer = ECL(model.parameters(), lr=lr)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_2584/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 20:18:40,249 - clearml.frameworks - INFO - Found existing registered model id=bbd65d869dea4025af46d264d3c7bdee [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-100-11.pt] reusing it.
Epoch 9 loss is 0.13289887250103694
Epoch 19 loss is 0.0969205175520467
Epoch 29 loss is 0.0876257468319937
Epoch 39 loss is 0.09244551632670642
Epoch 49 loss is 0.06913963450583839
Epoch 59 loss is 0.05862448278693276
Epoch 69 loss is 0.06090114262274447
Epoch 79 loss is 0.05708236092705054
Epoch 89 loss is 0.056327734121452935
Epoch 99 loss is 0.058758856743251306
Epoch 109 loss is 0.05308146111475717
Epoch 119 loss is 0.05663969716960362
Epoch 129 loss is 0.050931966043855297
Epoch 139 loss is 0.0470478870489573
Epoch 149 loss is 0.04390343749136136
Epoch 159 loss is 0.0413619943037058
Epoch 169 loss is 0.04258073464153097
Epoch 179 loss is 0.044157359217192584
Epoch 189 loss is 0.04075450701217185
Epoch 199 loss is 0.03822650540962055
Train Acc.:  0.97137119

In [ ]:
task.mark_completed()
task.close()